# Данные

In [4]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

corp_col = '#40ba21'


## Для анализа использован массив "RedWineQuality.csv".

In [7]:
df = pd.read_csv("RedWineQuality.csv", delimiter=";")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [ ]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

# Домашнее задание 10. Непараметрическая статистика.

## Средние значения и доверительные интервалы

In [15]:
import numpy as np
from scipy import stats

# Для alcohol
alcohol_mean = np.mean(df['alcohol'])
alcohol_ci = stats.t.interval(0.95, len(df)-1, loc=alcohol_mean, scale=stats.sem(df['alcohol']))
print( "Среднее значение: ", round( alcohol_mean, 2 ), " в доверительном интервала (", alcohol_ci, ")" )

Среднее значение:  10.42  в доверительном интервала ( (np.float64(10.370710444963722), np.float64(10.475255783929336)) )


### Вывод: с вероятностью 95% истинное среднее содержание алкоголя в генеральной совокупности вин находится между 10.37% и 10.47%

In [16]:
# Для quality
quality_mean = np.mean(df['quality'])
quality_ci = stats.t.interval(0.95, len(df)-1, loc=quality_mean, scale=stats.sem(df['quality']))
print( "Среднее значение: ", round( quality_mean, 2 ), " в доверительном интервала (", quality_ci, ")" )

Среднее значение:  5.64  в доверительном интервала ( (np.float64(5.596409964035299), np.float64(5.67563506410729)) )


### Вывод: С вероятностью 95% истинное среднее оценка качества в генеральной совокупности вин находится между 5.60% и 5.68%

In [19]:
# Shapiro-Wilk test
shapiro_alcohol = stats.shapiro(df['alcohol'])
print( shapiro_alcohol )

ShapiroResult(statistic=np.float64(0.9288390813054377), pvalue=np.float64(6.644056905730039e-27))


In [20]:
shapiro_quality = stats.shapiro(df['quality'])
print( shapiro_quality )

ShapiroResult(statistic=np.float64(0.8575895017309976), pvalue=np.float64(9.515084999430088e-36))


### Вывод: Обе переменные значимо отклоняются от нормального распределения.

In [21]:
group_low = df[df['quality'] < 6]['alcohol']
group_high = df[df['quality'] >= 6]['alcohol']

# Двухвыборочный t-тест
t_test = stats.ttest_ind(group_high, group_low, equal_var=False)
t_test

TtestResult(statistic=np.float64(19.78218499094004), pvalue=np.float64(1.139484979295024e-77), df=np.float64(1516.753137126026))

In [25]:
group_high.shape

(855,)

In [22]:
# Тест Манна-Уитни
mannwhitney = stats.mannwhitneyu(group_high, group_low)
mannwhitney

MannwhitneyuResult(statistic=np.float64(481313.0), pvalue=np.float64(2.0060593309032682e-70))

### Вывод: оба теста показывают статистически значимые различия средних. Поскольку распределение alcohol не является нормальным, тест Манна-Уитни более надёжен в данной ситуации.

In [23]:
def bootstrap_diff_means(data1, data2, n_bootstrap=10000):
    diffs = []
    for _ in range(n_bootstrap):
        sample1 = np.random.choice(data1, size=len(data1), replace=True)
        sample2 = np.random.choice(data2, size=len(data2), replace=True)
        diffs.append(np.mean(sample1) - np.mean(sample2))
    return np.percentile(diffs, [2.5, 97.5])

bootstrap_ci = bootstrap_diff_means(group_high, group_low)
bootstrap_ci

array([0.83672412, 1.01966384])

### Вывод: Содержание алкоголя в винах высокого качества статистически значимо выше, чем в винах низкого качества (разница составляет 0.84–1.02%).

